# **Principios SOLID en POO**

SOLID es un conjunto de cinco principios de diseño en **programación orientada a objetos (POO)** que buscan mejorar la calidad, mantenibilidad y escalabilidad del código. Cada letra representa un principio:

- **S - Single Responsibility Principle (SRP):** Cada clase debe tener una única responsabilidad o razón de cambio. Es decir, una clase debe encargarse de una sola cosa.

- **O - Open/Closed Principle (OCP):** El código debe estar abierto para la extensión, pero cerrado para la modificación. Esto significa que se deben poder agregar nuevas funcionalidades sin alterar el código existente, por ejemplo, usando **herencia** o **interfaces**.

- **L - Liskov Substitution Principle (LSP):** Los objetos de una subclase deben poder reemplazar a los objetos de su clase base sin alterar el funcionamiento del programa.

- **I - Interface Segregation Principle (ISP):** No se deben forzar a las clases a depender de interfaces que no usan. Es mejor tener varias **interfaces pequeñas y específicas** en lugar de una interfaz grande y genérica.

- **D - Dependency Inversion Principle (DIP):** El código debe depender de **abstracciones** y no de implementaciones concretas. Esto se logra mediante la **inyección de dependencias**, lo que facilita la reutilización y pruebas del código.


### 1. ***Single Responsibility Principle (SRP)***

✅ **Una clase debe tener una sola razón para cambiar.**

❌ **Violación del SRP**

Esta clase maneja tanto los datos del empleado como la impresión del reporte:

In [ ]:
import bcrypt # Libreria para cifrar contraseñas

class Ecommerce:
    
    """
    Clase que maneja tanto la lógica de negocio del ecommerce como la gestión de contraseñas.
    Esto viola el Principio de Responsabilidad Única (SRP).
    """
    
    def __init__(self):
        self.users = {}
        
    def register(self, username, password):
        """
        Registra un usuario y cifra la contraseña.
        Problema: La clase Ecommerce no debería encargarse del cifrado de contraseñas.
        """
        salt = bcrypt.gensalt()
        hashed_password = bcrypt.hashpw(password.encode(), salt) # 🚨 Rompe SRP
        self.users[username] = hashed_password
        print(f"Usuario: {username} registrado con exito")

#Crea una instancia y registra un usuario
ecommerce = Ecommerce()
ecommerce.register("Sergio", "Sapr1988")

        

Usuario: Sergio registrado con exito


#### ✅ ***Aplicando SRP***

Separamos la lógica de los datos y la impresión del reporte:

In [1]:
import bcrypt # Libreria para cifrar contraseñas

class PasswordManager:
    """Maneja el cifrado y verificación de contraseñas, separando responsabilidades."""
    def encrypt_password(self, password: str) -> str:
        salt = bcrypt.gensalt()
        return bcrypt.hashpw(password.encode(), salt)
    
    def verify_password():
        pass # Metodo pendiente de implementación
    
class Ecommerce:
    """Gestiona el registro de usuarios sin encargarse del cifrado, respetando SRP."""
    
    def __init__(self, password_manager: PasswordManager ):
        self.users = {} # Almacena usuarios con sus contraseñas cifradas
        self.password_manager = password_manager # Delegación de la gestión de contraseñas
        
    def register(self, username, password):
        """Registra un usuario con su contraseña cifrada mediante PasswordManager."""
        hashed_password = self.password_manager.encrypt_password(password)
        self.users[username] = hashed_password
        print(f"Usuario: {username} registrado con exito")

# Crear instancias y registrar usuarios 
passwordManager = PasswordManager()       
ecommerce = Ecommerce(passwordManager)
ecommerce.register("Sergio", "Sapr1988")

Usuario: Sergio registrado con exito


### ***2. Open/Closed Principle (OCP)***
✅ **El código debe ser abierto para extensión, pero cerrado para modificación.**

#### ❌ ***Violación del OCP***

Cada vez que agregamos un nuevo tipo de figura, modificamos la clase:

In [2]:
class CalculadoraDeAreas:
    """
    🚨 Esta clase viola el Principio de Abierto/Cerrado (OCP).
    Cada vez que queramos agregar una nueva figura, debemos modificar el código existente.
    """    
    def calcularArea(self, figura: str, **kwargs):
        if figura == "circulo":
            return 3.14 * (kwargs["radio"] ** 2)
        elif figura == "rectangulo":
            return kwargs["ancho"] * kwargs["alto"]
        # 🚨 Si agregamos más figuras, tenemos que modificar esta función, lo cual rompe OCP

# Uso de la calculadora        
calculadora = CalculadoraDeAreas()
print(f"El area del circulo es: {calculadora.calcularArea("circulo", radio=5)} m²")       
print(f"El area del rectangulo es: {calculadora.calcularArea("rectangulo", ancho=6, alto=4)} m²")       

El area del circulo es: 78.5 m²
El area del rectangulo es: 24 m²


#### ✅ ***Aplicando OCP***

Usamos herencia y polimorfismo en lugar de modificar la clase original.

In [3]:
from abc import ABC, abstractmethod

class FiguraGeometrica:
    # ✅ Aplicamos el Principio de Abierto/Cerrado (OCP) usando polimorfismo y abstracción
    
    @abstractmethod
    def calcularArea(self) -> float:
        pass # Cada figura implementará su propio cálculo de área
    
# ✅ Cada figura geométrica se define en su propia clase sin modificar el código existente    
class Circulo(FiguraGeometrica):
    def __init__(self, radio: float):
        self.radio = radio
        
    def calcularArea(self):
        return 3.14 * (self.radio ** 2)
    
class Rectangulo(FiguraGeometrica):
    def __init__(self, ancho: float, alto: float):
        self.ancho = ancho
        self.alto = alto

    def calcularArea(self):
        return self.ancho * self.alto
    
class Triangulo(FiguraGeometrica):
    def __init__(self, base: float, altura: float):
        self.base = base
        self.altura = altura
        
    def calcularArea(self):
        return (self.base * self.altura) / 2

# ✅ La CalculadoraDeAreas ahora puede aceptar cualquier nueva figura sin modificar su código
class CalculadoraAreasDeFiguras:
    def calcular(figura: FiguraGeometrica):
        return figura.calcularArea()
    
#Creamos una instancia de la calculadora
calculadora = CalculadoraAreasDeFiguras

#Crear instancias de las figuras
circulo = Circulo(5)      
rectangulo = Rectangulo(6, 4)      
triangulo = Triangulo(5, 8)

# Calculamos las areas de las figuras
print(f"El area del circulo es: {calculadora.calcular(circulo)} m²")      
print(f"El area del rectangulo es: {calculadora.calcular(rectangulo)} m²")      
print(f"El area del triangulo es: {calculadora.calcular(triangulo)} m²")      
            

El area del circulo es: 78.5 m²
El area del rectangulo es: 24 m²
El area del triangulo es: 20.0 m²


### ***3. Liskov Substitution Principle (LSP)***
✅ Las subclases deben poder reemplazar a sus clases base sin alterar el comportamiento.

#### ❌ ***Violación del LSP***
Supongamos que tenemos una clase Vehiculo con un método acelerar().
Luego, creamos una subclase Bicicleta, pero como las bicicletas no tienen motor, acelerar con una bicicleta no tiene sentido.

In [4]:
class Vehiculo:
    def acelerar(self):
        print("Aumento de velocidad")

class Coche(Vehiculo):
    def acelerar(self):
        print("El coche acelera con el motor")

# 🔴 Bicicleta hereda de Vehiculo, pero no debería        
class Bicicleta(Vehiculo):
    def acelerar(self):
        raise NotImplementedError("Las bicicletas no tiene acelerador")

# Funcion para acelerar los vehiculos    
def acelerarVehiculo(vehiculo: Vehiculo):
    vehiculo.acelerar()

# Creamos instancias de las subclases de Vehiculo    
coche = Coche()
bicicleta = Bicicleta()

acelerarVehiculo(coche)      # ✅ Correcto: "El coche acelera con el motor"
acelerarVehiculo(bicicleta)  # 🔴 Error: NotImplementedError
        

El coche acelera con el motor


NotImplementedError: Las bicicletas no tiene acelerador

#### ✅ ***Ejemplo corregido que respeta LSP***
Para solucionar el problema, separamos VehiculoConMotor y VehiculoSinMotor para que cada uno tenga métodos apropiados.

In [ ]:
from abc import ABC, abstractmethod

# ✅ Creamos una clase base abstracta para cualquier tipo de vehículo
class Vehiculo(ABC):
    
    @abstractmethod
    def moverse(self):
        pass

# ✅ Coche hereda de Vehiculo y puede implementar el método correctamente    
class Coche(Vehiculo):
    def moverse(self):
        print("El coche acelera con el motor")
        
# ✅ Bicicleta es un Vehiculo, pero implementa un método diferente (pedalear en lugar de acelerar)
class Bicicleta(Vehiculo):
    def moverse(self):
        print("La bicicleta se desplaza al pedalear")
        
def probrarMovimiento(vehiculo: Vehiculo):
    vehiculo.moverse()
    
# Creamos instancias de cada clase
coche = Coche()
bicicleta = Bicicleta()

probrarMovimiento(coche)      # ✅ "El coche acelera con el motor"
probrarMovimiento(bicicleta)  # ✅ "La bicicleta se desplaza al pedalear"


El coche acelera con el motor
La bicicleta se desplaza al pedalear


### Otro ejemplo de LSP aplicado

In [ ]:
from abc import ABC, abstractmethod

class Vehiculo(ABC):
    @abstractmethod
    def moverse(self):
        pass
    
class VehiculoConMotor(Vehiculo, ABC):
    @abstractmethod
    def acelerar(self):
        pass

class Bicicleta(Vehiculo):
    def moverse(self):
        print("La bicicleta se está moviendo")
        
    def pedalear(self):
        print("La bicicleta se desplaza al pedalear")

class Coche(VehiculoConMotor):
    def moverse(self):
        print("El coche se está moviendo")
        
    def acelerar(self):
        print("El coche acelera con el motor")

class Moto(VehiculoConMotor):
    def moverse(self):
        print("La moto se está moviendo")
        
    def acelerar(self):
        print("La moto acelera con el motor")

# Función para probar el movimiento de cualquier Vehiculo
def probar_movimiento(vehiculo: Vehiculo):
    vehiculo.moverse()

# Función específica para probar aceleración en vehículos con motor
def probar_aceleracion(vehiculo: VehiculoConMotor):
    vehiculo.acelerar()

# Instancias
coche = Coche()
bicicleta = Bicicleta()
moto = Moto()

# Probar movimiento (respeta LSP)
probar_movimiento(coche)      # ✅ "El coche se está moviendo"
probar_movimiento(bicicleta)  # ✅ "La bicicleta se está moviendo"
probar_movimiento(moto)       # ✅ "La moto se está moviendo"

# Probar aceleración (respeta LSP)
probar_aceleracion(coche)  # ✅ "El coche acelera con el motor"
probar_aceleracion(moto)   # ✅ "La moto acelera con el motor"

# Probar pedaleo
bicicleta.pedalear()

El coche se está moviendo
La bicicleta se está moviendo
La moto se está moviendo
El coche acelera con el motor
La moto acelera con el motor
La bicicleta se desplaza al pedalear


#### ***✅ ¿Por qué este código respeta LSP?***
✔️ Ahora Bicicleta y Coche pueden sustituir correctamente a Vehiculo, sin generar errores.

✔️ Cada clase tiene solo los métodos que realmente necesita, sin implementar métodos inválidos.

✔️ El programa sigue funcionando si cambiamos un Vehiculo por otro.

### ***5. Dependency Inversion Principle (DIP)***
✅ **Depende de abstracciones, no de implementaciones concretas.**

#### ❌ ***Ejemplo que viola DIP***
Aquí tenemos un sistema donde la clase Notificador depende directamente de la clase EmailService.

In [ ]:
class EmailService:
    def enviarEmail(self, mensaje: str):
        print(f"Enviar email: {mensaje}")
        
class Notificar:
    def __init__(self):
        self.emailServicio = EmailService()
        
    def notificar(self, mensaje: str):
        self.emailServicio.enviarEmail(mensaje)

notificador = Notificar()

notificador.notificar("Hola somos Dev Senior")  

Enviar email: Hola somos Dev Senior


In [ ]:
from abc import ABC, abstractmethod

class Notificacion(ABC):
    @abstractmethod
    def enviar(self, mensaje: str):
        pass
    
class EmailService(Notificacion):
    def enviar(self, mensaje: str):
        print(f"Enviar email: {mensaje}")
        
class WhatsappService(Notificacion):
    def enviar(self, mensaje: str):
        print(f"Enviar Whastapp: {mensaje}")
        
class SMSService(Notificacion):
    def enviar(self, mensaje: str):
        print(f"Enviar SMS: {mensaje}")
        
class Notificador:
    def __init__(self, servicio: Notificacion):
        self.servicio = servicio
        
    def notificar(self, mensaje: str):
        self.servicio.enviar(mensaje)
        
class NotificadorModificable:
    def __init__(self, notificador: Notificador):
        self.notificacion = notificador
        self.mensaje = None
        
    def notificar(self, mensaje: str):
        self.mensaje = mensaje
        self.notificacion.notificar(mensaje)
        
    def modificarMensaje(self, nuevoMensaje: str):
        if self.mensaje is None:
            print("No hay mensaje previo para modificar")
            return
        self.mensaje = nuevoMensaje
        print(f"Mensaje modificado a: {self.mensaje}")
        self.notificacion.notificar(self.mensaje)

     
emailNotificador = NotificadorModificable(Notificador(EmailService()))
emailNotificador.notificar("Hola, somos DevSenior desde un email")

whatsappNotificador = Notificador(WhatsappService())
whatsappNotificador.notificar("Hola, somos DevSenior desde Whatsapp")

sms_notificador = Notificador(SMSService())
sms_notificador.notificar("Hola, somos DevSenior desde SMS")

emailNotificador.notificar("Hola, somos DevSenior desde un Email, gracias por elegirnos")

Enviar email: Hola, somos DevSenior desde un email
Enviar Whastapp: Hola, somos DevSenior desde Whatsapp
Enviar SMS: Hola, somos DevSenior desde SMS
Enviar email: Hola, somos DevSenior desde un Email, gracias por elegirnos


#### ✅ ***¿Por qué este código respeta DIP?***
✔️ Notificador ya no depende de una clase concreta (EmailService o SMSService), sino de la interfaz INotificador.

✔️ Podemos agregar nuevas implementaciones (ej. WhatsApp, Telegram, etc.) sin modificar Notificador.

✔️ El código es más flexible y abierto a la extensión sin afectar las clases existentes.

### ***Conclusión sobre SOLID y su importancia para un desarrollador***

Los principios **SOLID** son fundamentales para escribir código **limpio, mantenible y escalable**. Cada uno de estos principios aborda problemas comunes en el desarrollo de software, permitiendo que las aplicaciones sean más **modulares, reutilizables** y fáciles de extender sin necesidad de modificar el código existente.

#### ***Importancia de SOLID para un desarrollador:***

- **Facilita el mantenimiento del código:**
  Aplicar SOLID reduce la complejidad y facilita la localización y corrección de errores sin afectar otras partes del sistema.

- **Promueve la reutilización de código:**
  Diseñar clases y módulos con responsabilidades bien definidas permite reutilizar componentes en diferentes partes de un proyecto o en futuros desarrollos.

- **Mejora la escalabilidad del software:**
  Un código bien estructurado según SOLID es más fácil de extender con nuevas funcionalidades sin necesidad de modificar el código existente, evitando la introducción de nuevos errores.

- **Reduce el acoplamiento y mejora la modularidad:**
  Siguiendo SOLID, cada clase y módulo depende solo de lo estrictamente necesario, evitando dependencias innecesarias y facilitando la prueba y depuración del código.

- **Fomenta las buenas prácticas de diseño:**
  Aplicar SOLID ayuda a adoptar un enfoque de desarrollo más profesional, siguiendo estándares que facilitan el trabajo en equipo y la colaboración en proyectos grandes.

**SOLID no es solo un conjunto de reglas, sino una guía para escribir mejor código.**
Todo desarrollador que quiera crear software de calidad debería aplicar estos principios para lograr sistemas más **robustos, flexibles y fáciles de mantener**. 🚀